In [228]:
import requests
import csv
import datetime
from datetime import timedelta
import pandas as pd
import json
import matplotlib.path as mplPath
from tqdm import tqdm
import folium
import os
from IPython.display import display
from folium.features import DivIcon

# Load Comptages routiers permanents

In [190]:
current_date = datetime.datetime.now().replace(microsecond=0).isoformat()
current_date += 'Z'
next_day = datetime.datetime.now() - timedelta(days=1)
next_day = str(next_day.replace(microsecond=0)) + 'Z'
next_day = next_day.replace(" ", "T")
query = 't_1h: [' + current_date + ' TO ' + next_day + ']'


payload = {'limit': 10000,
           'q': query,
           'sort': '-t_1h',
           'facet': 't_1h'
          }

res1 = requests.get('https://opendata.paris.fr/api/v2/catalog/datasets/comptages-routiers-permanents/exports/csv/',
                  params=payload)

decoded_content = res1.content.decode('utf-8')

cr = csv.reader(decoded_content.splitlines(), delimiter=';')
my_list = list(cr)
df = pd.DataFrame(my_list)
df = df.rename(columns=df.iloc[0]).drop(0)
df = df.reset_index(drop=True)
print(df.shape)
df.head(5)

(10000, 15)


,iu_ac,libelle,t_1h,q,k,etat_trafic,iu_nd_amont,libelle_nd_amont,iu_nd_aval,libelle_nd_aval,etat_barre,date_debut,date_fin,geo_point_2d,geo_shape
0,4541,Bd_Lannes,2021-12-15T23:00:00+00:00,169.0,0.58611,Fluide,2423,Bd_Lannes-Longchamp,2422,Lannes-Piscine,Invalide,2005-01-01,2019-06-01,"48.8692906671, 2.2737212675","{""coordinates"": [[2.2741526711, 48.8698087375]..."
1,5641,Souterrain_Dauphine,2021-12-15T23:00:00+00:00,169.0,,Inconnu,2964,Amiral_Bruix-Souterrain_Dauphine,2423,Bd_Lannes-Longchamp,Invalide,2005-01-01,2019-06-01,"48.8718594869, 2.27560783241","{""coordinates"": [[2.2772596514, 48.8737414845]..."
2,4540,Bd_Lannes,2021-12-15T23:00:00+00:00,,,Inconnu,2424,Pte_Dauphine-Lannes,2423,Bd_Lannes-Longchamp,Invalide,2005-01-01,2019-06-01,"48.8703893526, 2.27446288109","{""coordinates"": [[2.2747730911, 48.8709699677]..."
3,4538,Bd_Lannes,2021-12-15T23:00:00+00:00,163.0,,Inconnu,2422,Lannes-Piscine,2423,Bd_Lannes-Longchamp,Invalide,2005-01-01,2019-06-01,"48.8692523427, 2.27390750662","{""coordinates"": [[2.2735152314, 48.8687388812]..."
4,5645,Pte_Dauphine,2021-12-15T23:00:00+00:00,,,Inconnu,2963,Pte_Dauphine-Marechal_Fayolle-PI,2424,Pte_Dauphine-Lannes,Invalide,2005-01-01,2019-06-01,"48.871087067, 2.27446520338","{""coordinates"": [[2.2741573157, 48.8712041664]..."


# Load Quartier Paris

In [212]:
payload2 = {'facet': 'l_qu'}

res2 = requests.get('https://opendata.paris.fr/api/v2/catalog/datasets/quartier_paris/exports/csv/',
                  params=payload2)

decoded_content2 = res2.content.decode('utf-8')

cr2 = csv.reader(decoded_content2.splitlines(), delimiter=';')
my_list2 = list(cr2)
df2 = pd.DataFrame(my_list2)
df2 = df2.rename(columns=df2.iloc[0]).drop(0)
df2 = df2.reset_index(drop=True)
print(df2.shape)
df2.head(5)

(80, 10)


,n_sq_qu,c_qu,c_quinsee,l_qu,c_ar,n_sq_ar,perimetre,surface,geom_x_y,geom
0,750000009,9,7510301,Arts-et-Métiers,3,750000003,2482.4604529,318087.74045393,"48.8664702895, 2.35708313106","{""coordinates"": [[[2.360209979547445, 48.86519..."
1,750000044,44,7511104,Sainte-Marguerite,11,750000011,4591.31079852,929609.24169048,"48.852096507, 2.3887648336","{""coordinates"": [[[2.3962365762994082, 48.8541..."
2,750000068,68,7511704,Epinettes,17,750000017,5171.70645067,1378544.9478026,"48.8949426538, 2.32111874371","{""coordinates"": [[[2.325914214090841, 48.88853..."
3,750000072,72,7511804,La Chapelle,18,750000018,5261.43659644,1347544.5496766,"48.8940121696, 2.36438677903","{""coordinates"": [[[2.365803615285393, 48.88554..."
4,750000073,73,7511901,Villette,19,750000019,5191.01882968,1285705.01238933,"48.8876610888, 2.37446821213","{""coordinates"": [[[2.370498649617153, 48.87819..."


In [213]:
for i in range(len(df2)):
    df2.loc[i, 'geom_custom'] = str(json.loads(df2.loc[i, 'geom'])['coordinates'])

In [214]:
df2

,n_sq_qu,c_qu,c_quinsee,l_qu,c_ar,n_sq_ar,perimetre,surface,geom_x_y,geom,geom_custom
0,750000009,9,7510301,Arts-et-Métiers,3,750000003,2482.4604529,318087.74045393,"48.8664702895, 2.35708313106","{""coordinates"": [[[2.360209979547445, 48.86519...","[[[2.360209979547445, 48.86519024025307], [2.3..."
1,750000044,44,7511104,Sainte-Marguerite,11,750000011,4591.31079852,929609.24169048,"48.852096507, 2.3887648336","{""coordinates"": [[[2.3962365762994082, 48.8541...","[[[2.3962365762994082, 48.854154588386436], [2..."
2,750000068,68,7511704,Epinettes,17,750000017,5171.70645067,1378544.9478026,"48.8949426538, 2.32111874371","{""coordinates"": [[[2.325914214090841, 48.88853...","[[[2.325914214090841, 48.88853299519872], [2.3..."
3,750000072,72,7511804,La Chapelle,18,750000018,5261.43659644,1347544.5496766,"48.8940121696, 2.36438677903","{""coordinates"": [[[2.365803615285393, 48.88554...","[[[2.365803615285393, 48.885544295479654], [2...."
4,750000073,73,7511901,Villette,19,750000019,5191.01882968,1285705.01238933,"48.8876610888, 2.37446821213","{""coordinates"": [[[2.370498649617153, 48.87819...","[[[2.370498649617153, 48.87819784234086], [2.3..."
...,...,...,...,...,...,...,...,...,...,...,...
75,750000015,15,7510403,Arsenal,4,750000004,2878.55965556,487264.93707154,"48.851585175, 2.36476795387","{""coordinates"": [[[2.368512371393433, 48.85573...","[[[2.368512371393433, 48.85573412813671], [2.3..."
76,750000023,23,7510603,Notre-Dame-des-Champs,6,750000006,4559.98977263,861307.03782615,"48.846427594, 2.32735687823","{""coordinates"": [[[2.336756578897731, 48.84013...","[[[2.336756578897731, 48.84013064406058], [2.3..."
77,750000057,57,7511501,Saint-Lambert,15,750000015,6928.79207234,2829201.55405581,"48.8342936284, 2.29691997445","{""coordinates"": [[[2.304248796649131, 48.84040...","[[[2.304248796649131, 48.84040838273815], [2.3..."
78,750000067,67,7511703,Batignolles,17,750000017,5832.69002632,1441710.87038814,"48.8884815139, 2.31385616901","{""coordinates"": [[[2.317210277038245, 48.89025...","[[[2.317210277038245, 48.890259814260965], [2...."


# Unique rue avec geopoint

In [215]:
# notre df avec les geopoints pour calculer le quartier
df_iu_geo = df[['iu_ac', 'geo_point_2d']].drop_duplicates()
df_iu_geo

,iu_ac,geo_point_2d
0,4541,"48.8692906671, 2.2737212675"
1,5641,"48.8718594869, 2.27560783241"
2,4540,"48.8703893526, 2.27446288109"
3,4538,"48.8692523427, 2.27390750662"
4,5645,"48.871087067, 2.27446520338"
...,...,...
2977,6559,"48.8787971472, 2.35492982074"
2978,1567,"48.8798733872, 2.35276664746"
2979,1564,"48.8789308863, 2.35374108882"
2980,1566,"48.8799097296, 2.35288150374"


# Calcule du quartier selon le geopoint

In [216]:
def is_in_poly(geo_point, poly):
    try:
        poly1 = json.loads(poly)['coordinates']
        poly1 = [[i[1],i[0]] for i in poly1[0]]
        poly_path = mplPath.Path(poly1)
        tmp =  geo_point.split(', ')
        x1 =(float(tmp[0]), float(tmp[1]))
        return poly_path.contains_point(x1)
    except:
        return False

In [217]:
for i in tqdm(range(len(df_iu_geo))):
    for j in range(len(df2)):
        if is_in_poly(df_iu_geo.loc[i].geo_point_2d, df2.loc[j].geom):
            df_iu_geo.loc[i, 'c_qu'] = df2.loc[j, 'c_qu']
            break

100%|██████████| 2982/2982 [00:44<00:00, 67.16it/s]


In [218]:
df_iu_geo

,iu_ac,geo_point_2d,c_qu
0,4541,"48.8692906671, 2.2737212675",63
1,5641,"48.8718594869, 2.27560783241",63
2,4540,"48.8703893526, 2.27446288109",63
3,4538,"48.8692523427, 2.27390750662",63
4,5645,"48.871087067, 2.27446520338",63
...,...,...,...
2977,6559,"48.8787971472, 2.35492982074",37
2978,1567,"48.8798733872, 2.35276664746",37
2979,1564,"48.8789308863, 2.35374108882",37
2980,1566,"48.8799097296, 2.35288150374",37


In [219]:
df_merged = df.merge(df_iu_geo[['iu_ac', 'c_qu']], on='iu_ac', how='left')
df_merged = df_merged.merge(df2[['c_qu', 'l_qu', 'c_ar', 'geom', 'geom_x_y']], on='c_qu', how='left' )
df_merged = df_merged[df_merged.k != ''].reset_index(drop=True)
df_merged = df_merged.astype({"k": float})

In [220]:
#df_merged = df_merged.groupby(['t_1h', 'c_qu', 'c_ar'])['k'].mean().to_frame().reset_index(drop=False).merge(df2[['c_qu', 'geom_custom']], on='c_qu', how='left')

In [221]:
#df_merged.to_csv('notre_fichier.csv', index=False)

In [222]:
df_test = df_merged.groupby(['t_1h', 'c_qu', 'c_ar', 'geom', 'geom_x_y'])['k'].mean().to_frame().reset_index(drop=False)

In [223]:
for i in range(len(df_test)):
    k = df_test.loc[i, 'k']
    if 0 <= k and k < 15 :
        df_test.loc[i, 'risk'] = 0
    elif 15 <= k and k < 30:
        df_test.loc[i, 'risk'] = 1
    elif 30 <= k and k < 50:
        df_test.loc[i, 'risk'] = 2
    else:
        df_test.loc[i, 'risk'] = 3

In [278]:
for j in df_test.t_1h.unique():
    loc = 'Paris : ' + str(j).replace('T', ' ').split('+')[0]
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(loc)  
    paris = folium.Map(location = [48.856578, 2.351828], 
                       zoom_start = 12
                      )
    df_tmp = df_test[df_test['t_1h'] == j]
    df_tmp = df_tmp.reset_index(drop=False)
    for i in range(len(df_tmp)):
        if df_tmp.loc[i, 'risk'] == 0:
            paris.choropleth(geo_data = df_tmp.geom[i],
                             fill_color ='green')
        elif df_tmp.loc[i, 'risk'] == 1:
            paris.choropleth(geo_data = df_tmp.geom[i],
                             fill_color ='yellow')
        elif df_tmp.loc[i, 'risk'] == 2:
            paris.choropleth(geo_data = df_tmp.geom[i],
                                 fill_color ='orange')
        else:
            paris.choropleth(geo_data = df_tmp.geom[i],
                                 fill_color ='red')
    paris.get_root().html.add_child(folium.Element(title_html))
    
    df_tmpp = df_test[['geom_x_y', 'c_qu']].drop_duplicates().reset_index(drop=True)
    for label in range(len(df_tmpp)):
        folium.map.Marker(
        [float(i) for i in df_tmpp.loc[label, 'geom_x_y'].split(", ")],
        icon=DivIcon(
            html='<div style="font-size: 12pt">'+str(df_tmpp.loc[label, 'c_qu'])+'</div>',
            )
        ).add_to(paris)
        print("toto")
    
    
    
    
    display(paris)

toto


toto


toto


toto


toto


...

toto


toto


toto


toto


toto


In [225]:
df_test

,t_1h,c_qu,c_ar,geom,geom_x_y,k,risk
0,2021-12-15T20:00:00+00:00,1,1,"{""coordinates"": [[[2.344593389828428, 48.85404...","48.8606501352, 2.33491032928",11.781429,0.0
1,2021-12-15T20:00:00+00:00,11,3,"{""coordinates"": [[[2.368479720528894, 48.85583...","48.8591924127, 2.36320505733",4.724450,0.0
2,2021-12-15T20:00:00+00:00,13,4,"{""coordinates"": [[[2.352623485962105, 48.85490...","48.8585213723, 2.35166696714",9.400372,0.0
3,2021-12-15T20:00:00+00:00,14,4,"{""coordinates"": [[[2.363764600226478, 48.85567...","48.8557186509, 2.35816233385",8.792777,0.0
4,2021-12-15T20:00:00+00:00,15,4,"{""coordinates"": [[[2.368512371393433, 48.85573...","48.851585175, 2.36476795387",9.665278,0.0
...,...,...,...,...,...,...,...
310,2021-12-15T23:00:00+00:00,78,20,"{""coordinates"": [[[2.413813296947322, 48.87081...","48.8710347391, 2.40617153015",3.587187,0.0
311,2021-12-15T23:00:00+00:00,79,20,"{""coordinates"": [[[2.395358818714641, 48.85669...","48.863719484, 2.39527282076",3.440580,0.0
312,2021-12-15T23:00:00+00:00,8,2,"{""coordinates"": [[[2.351518483670822, 48.86442...","48.8671501183, 2.35008019041",9.157723,0.0
313,2021-12-15T23:00:00+00:00,80,20,"{""coordinates"": [[[2.4160210576126833, 48.8467...","48.854760283, 2.40743032824",3.337166,0.0


In [ ]:

# Access the centroid attribute of each polygon
df['centroid'] = df.centroid

In [236]:
toto = [float(i) for i in df_test.geom_x_y.unique()[0].split(", ")]  df_test.geom_x_y.unique()[0].split(", ")


In [240]:
res = [float(i) for i in toto]

In [241]:
res


[48.8606501352, 2.33491032928]

In [246]:
df_test

,t_1h,c_qu,c_ar,geom,geom_x_y,k,risk
0,2021-12-15T20:00:00+00:00,1,1,"{""coordinates"": [[[2.344593389828428, 48.85404...","48.8606501352, 2.33491032928",11.781429,0.0
1,2021-12-15T20:00:00+00:00,11,3,"{""coordinates"": [[[2.368479720528894, 48.85583...","48.8591924127, 2.36320505733",4.724450,0.0
2,2021-12-15T20:00:00+00:00,13,4,"{""coordinates"": [[[2.352623485962105, 48.85490...","48.8585213723, 2.35166696714",9.400372,0.0
3,2021-12-15T20:00:00+00:00,14,4,"{""coordinates"": [[[2.363764600226478, 48.85567...","48.8557186509, 2.35816233385",8.792777,0.0
4,2021-12-15T20:00:00+00:00,15,4,"{""coordinates"": [[[2.368512371393433, 48.85573...","48.851585175, 2.36476795387",9.665278,0.0
...,...,...,...,...,...,...,...
310,2021-12-15T23:00:00+00:00,78,20,"{""coordinates"": [[[2.413813296947322, 48.87081...","48.8710347391, 2.40617153015",3.587187,0.0
311,2021-12-15T23:00:00+00:00,79,20,"{""coordinates"": [[[2.395358818714641, 48.85669...","48.863719484, 2.39527282076",3.440580,0.0
312,2021-12-15T23:00:00+00:00,8,2,"{""coordinates"": [[[2.351518483670822, 48.86442...","48.8671501183, 2.35008019041",9.157723,0.0
313,2021-12-15T23:00:00+00:00,80,20,"{""coordinates"": [[[2.4160210576126833, 48.8467...","48.854760283, 2.40743032824",3.337166,0.0


In [253]:
df_test[['geom_x_y', 'c_qu']].drop_duplicates().reset_index(drop=True)

,geom_x_y,c_qu
0,"48.8606501352, 2.33491032928",1
1,"48.8591924127, 2.36320505733",11
2,"48.8585213723, 2.35166696714",13
3,"48.8557186509, 2.35816233385",14
4,"48.851585175, 2.36476795387",15
...,...,...
75,"48.863887392, 2.36312330099",10
76,"48.862557245, 2.35485151825",12
77,"48.8799343388, 2.33285019656",33
78,"48.8670185906, 2.32858166493",4


In [260]:
[float(i) for i in df_tmpp.loc[0, 'geom_x_y'].split(", ")]

[48.8606501352, 2.33491032928]

In [259]:
df_tmpp.loc[0, 'geom_x_y']

'48.8606501352, 2.33491032928'

In [279]:
folium.map.Marker

folium.map.Marker

In [280]:
help(folium.map.Marker)

Help on class Marker in module folium.map:

class Marker(branca.element.MacroElement)
 |  Marker(location=None, popup=None, tooltip=None, icon=None, draggable=False, **kwargs)
 |  
 |  Create a simple stock Leaflet marker on the map, with optional
 |  popup text or Vincent visualization.
 |  
 |  Parameters
 |  ----------
 |  location: tuple or list
 |      Latitude and Longitude of Marker (Northing, Easting)
 |  popup: string or folium.Popup, default None
 |      Label for the Marker; either an escaped HTML string to initialize
 |      folium.Popup or a folium.Popup instance.
 |  tooltip: str or folium.Tooltip, default None
 |      Display a text when hovering over the object.
 |  icon: Icon plugin
 |      the Icon plugin to use to render the marker.
 |  draggable: bool, default False
 |      Set to True to be able to drag the marker around the map.
 |  
 |  Returns
 |  -------
 |  Marker names and HTML in obj.template_vars
 |  
 |  Examples
 |  --------
 |  >>> Marker(location=[45.5,